# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
import umap

import torch
from torch import nn
import torch.nn.functional as F

from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood

In [ ]:
import matplotlib.pyplot as plt

## Dataset